# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
os.getcwd()

'/Users/nicholehartz/Documents/DataScience/Projects/Guided Capstone 1'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [3]:
path="data"
os.chdir(path)

In [4]:
os.listdir()

['.DS_Store', 'step2_output.csv', 'step3_output.csv']

In [5]:
ski_data = pd.read_csv('step3_output.csv')

In [6]:
ski_data.head()

,Unnamed: 0,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,...,30.0,30.000000,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,1
1,4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,...,800.0,80.000000,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,0
2,5,Yosemite Ski & Snowboard Area,California,7800,600,0,0.0,0,0,0,...,88.0,174.873239,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,0
3,8,Boreal Mountain Resort,California,7700,500,0,0.0,0,1,1,...,380.0,200.000000,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,0
4,9,Dodge Ridge,California,8200,1600,0,0.0,0,0,1,...,862.0,174.873239,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,0


In [7]:
ski_data.drop(columns='Unnamed: 0', inplace=True)

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [8]:
ski_data = pd.concat([ski_data, pd.get_dummies(ski_data['state'])], axis=1).drop(['state'], axis=1)

In [9]:
ski_data.head()

,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0
2,Yosemite Ski & Snowboard Area,7800,600,0,0.0,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
3,Boreal Mountain Resort,7700,500,0,0.0,0,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
4,Dodge Ridge,8200,1600,0,0.0,0,0,1,2,5,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [10]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = ski_data.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = ski_data['AdultWeekend']

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [11]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [12]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [13]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

In [14]:
y_pred

array([ 6.94421902e+01,  5.41514187e+01,  4.93862820e+01,  4.53408718e+01,
        5.76772976e+01,  5.78123074e+01,  5.69490261e+01,  8.15339870e+01,
        4.57026882e+01,  6.27468777e+01,  8.15144558e+01,  7.99558620e+01,
        4.66504421e+01,  5.67498074e+01,  6.21694851e+01,  5.32288113e+01,
        9.49382839e+01,  3.35515652e+01,  4.28015652e+01,  3.90476589e+01,
        4.86511745e+01,  4.88372097e+01,  6.00095730e+01,  5.85183620e+01,
        3.26783962e+01,  7.41335964e+01,  5.92517605e+01,  3.23386745e+01,
        5.77083035e+01,  5.31443386e+01,  5.94456081e+01,  8.49504910e+01,
        9.13728542e+01,  7.01414089e+01,  6.47083035e+01,  6.79011745e+01,
       -1.69109660e+12,  5.52459011e+01,  6.21448269e+01,  5.10847683e+01,
        5.10481472e+01,  3.38123074e+01,  6.69973660e+01,  3.36570339e+01])

## Review Model Outcomes — Iterate over additional models as needed

In [15]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [16]:
import sklearn.metrics
print('Explained Variance Score: ', explained_variance_score(y_test, y_pred))
print('Mean Absoulte Error: ', mean_absolute_error(y_test, y_pred))

Explained Variance Score:  -3.362018246443744e+20
Mean Absoulte Error:  38434013594.49269


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [17]:
print(lm.intercept_)

-9663409074.23945


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [18]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by='Coefficient', ascending=False).head(10)

,Coefficient
trams,2.876074e+13
fastEight,8.178761e+12
fastSixes,6.981885e+12
total_chairs,6.363051e+12
double,3.476605e+12
surface,3.334031e+12
triple,2.850098e+12
quad,1.624559e+12
fastQuads,1.359385e+12
New York,5.380348e+11


In [19]:
X.describe()

,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
count,175.00000,175.000000,175.0,175.0,175.0,175.000000,175.000000,175.000000,175.000000,175.000000,...,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,3707.12000,856.908571,0.0,0.0,0.0,0.240000,0.531429,1.194286,1.760000,2.245714,...,0.005714,0.005714,0.005714,0.017143,0.045714,0.011429,0.022857,0.017143,0.057143,0.028571
std,3245.48921,591.797770,0.0,0.0,0.0,0.586829,0.701301,1.230350,1.500804,1.439257,...,0.075593,0.075593,0.075593,0.130176,0.209464,0.106597,0.149877,0.130176,0.232781,0.167077
min,315.00000,175.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1250.00000,400.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2000.00000,652.000000,0.0,0.0,0.0,0.000000,0.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6575.00000,1156.000000,0.0,0.0,0.0,0.000000,1.000000,2.000000,2.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,12075.00000,2610.000000,0.0,0.0,0.0,2.000000,2.000000,5.000000,6.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [20]:
X2 = X[X.columns[:23]] 

In [21]:
scaler2 = preprocessing.StandardScaler().fit(X2)
X_scaled2 = scaler2.transform(X2) 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_scaled2, y, test_size=0.25, random_state=1)

In [22]:
model2 = lm.fit(X_train2,y_train2)

In [23]:
y_pred2 = model2.predict(X_test2)

In [24]:
print('Explained Variance Score: ', explained_variance_score(y_test2, y_pred2))
print('Mean Absoulte Error: ', mean_absolute_error(y_test2, y_pred2))

Explained Variance Score:  0.6819688787945681
Mean Absoulte Error:  6.424752625022816


In [25]:
print(lm.intercept_)

56.368945127197904


In [26]:
pd.DataFrame(abs(lm.coef_), X2.columns, columns=['Coefficient']).sort_values(by='Coefficient', ascending=False).head(10)

,Coefficient
AdultWeekday,10.281469
clusters,3.538121
daysOpenLastYear,2.331476
projectedDaysOpen,2.262525
TerrainParks,2.097151
vertical_drop,2.020495
quad,1.621398
LongestRun_mi,1.477068
surface,1.447687
SkiableTerrain_ac,1.221776


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [27]:
X3 = X2.drop(['summit_elev'], axis=1)

In [28]:
scaler3 = preprocessing.StandardScaler().fit(X3)
X_scaled3 = scaler3.transform(X3) 
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_scaled3, y, test_size=0.25, random_state=1)

In [29]:
model3 = lm.fit(X_train3,y_train3)

In [30]:
y_pred3 = model3.predict(X_test3)

In [31]:
print('Explained Variance Score: ', explained_variance_score(y_test3, y_pred3))
print('Mean Absoulte Error: ', mean_absolute_error(y_test3, y_pred3))

Explained Variance Score:  0.6791734823136522
Mean Absoulte Error:  6.460400355819531


In [32]:
print(lm.intercept_)

56.36883309501812


In [34]:
pd.DataFrame(abs(lm.coef_), X3.columns, columns=['Coefficient']).sort_values(by='Coefficient', ascending=False).head(10)

,Coefficient
AdultWeekday,10.298351
clusters,3.385266
daysOpenLastYear,2.356415
projectedDaysOpen,2.283904
vertical_drop,2.116992
TerrainParks,2.102350
quad,1.640734
surface,1.475546
LongestRun_mi,1.462194
SkiableTerrain_ac,1.169191


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection:

Model 3